In [1]:
import torchvision
import torch

In [3]:
torchvision.__version__

'0.11.1'

In [4]:
torch.__version__

'1.10.0'

In [17]:
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Lambda
transform = Compose([
    ToTensor(),
    # Lambda(lambda image: image / 255),
    Lambda(lambda image: image.view(784))
])
data_train = MNIST(root="./",download=True,train=True,transform=transform)
data_test = MNIST(root="./",download=True,train=False,transform=transform)

In [19]:
data_train[0][0].shape

torch.Size([784])

In [20]:
data_train[0][1]

5

In [42]:
from torch import nn, optim
import torch
def get_default_device():
    return torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def to_t(tensor,device=get_default_device()):
    return tensor.to(device)

class MNISTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(784,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
        self.loss = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters())
        self.to(get_default_device())
    def forward(self,X):
        return self.layers(X)
    
    def predict(self,X):
        with torch.no_grad():
            return torch.argmax(self.forward(X),axis=-1)
    def fit(self,X,Y):
        self.optimizer.zero_grad()
        y_pred = self.forward(X)
        loss = self.loss(y_pred,Y)
        loss.backward()
        self.optimizer.step()
        return loss.item()

In [43]:
mnist_model = MNISTModel()

In [44]:
from torch.utils.data import DataLoader
BATCH_SIZE = 16
dataloader_train = DataLoader(data_train,batch_size=BATCH_SIZE,shuffle=True)
dataloader_test = DataLoader(data_test,batch_size=BATCH_SIZE,shuffle=True)

In [45]:
from tqdm import tqdm
EPOCHS = 5
for i in range(EPOCHS):
    total_loss = 0
    for r_xs, r_ys in tqdm(dataloader_train,desc=f"FITTING EPOCH {i}"):
        xs, ys = to_t(r_xs), to_t(r_ys)
        total_loss += mnist_model.fit(xs,ys)
    total_loss /= len(dataloader_train)
    print(f"EPOCH {i}: {total_loss:.4f}")

FITTING EPOCH 1:   1%|▍                                                                    | 24/3750 [00:00<00:15, 233.81it/s]

EPOCH 0: 0.1954


FITTING EPOCH 2:   1%|▍                                                                    | 24/3750 [00:00<00:15, 237.95it/s]

EPOCH 1: 0.0873


FITTING EPOCH 3:   1%|▍                                                                    | 26/3750 [00:00<00:14, 253.07it/s]

EPOCH 2: 0.0637


FITTING EPOCH 4:   1%|▍                                                                    | 22/3750 [00:00<00:17, 213.58it/s]

EPOCH 3: 0.0485


FITTING EPOCH 4: 100%|███████████████████████████████████████████████████████████████████| 3750/3750 [00:15<00:00, 243.47it/s]

EPOCH 4: 0.0407


In [46]:
correct = 0
for r_xs, r_ys in dataloader_test:
    # print(mnist_model.predict(xs))
    xs, ys = to_t(r_xs), to_t(r_ys)
    y_pred = mnist_model.predict(xs)
    correct += (ys == y_pred).sum()
acc = correct / (len(dataloader_test) * BATCH_SIZE)
print(f"ACCURACY: {acc}")

ACCURACY: 0.976699948310852


In [39]:
len(dataloader_test)

625

In [41]:
625 * 16

10000